In [1]:
from appgeopy import *
from my_packages import *

In [2]:
input_fpath = r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_003\14_POST-PROCESSING\ras2pnt_MODIFIED_oChoushui_CUMDISP_LOS_mm.pkl"
df = pd.read_pickle(input_fpath, compression="zip")
df.head(3)

,PointKey,X_TWD97,Y_TWD97,N20220407,N20220419,N20220501,N20220513,N20220525,N20220606,N20220618,...,N20240115,N20240127,N20240208,N20240220,N20240303,N20240315,N20240327,N20240408,N20240420,geometry
0,X237393Y2705479,237393.759851,2.705479e+06,-7.5685,-9.0007,0.5446,17.2804,7.3864,8.0557,13.9074,...,84.8412,67.5040,62.9011,84.7707,93.3258,95.0676,98.7330,102.6851,119.1568,POINT (237393.760 2705479.209)
1,X237382Y2705465,237382.784009,2.705465e+06,-7.2381,-8.7758,0.8092,17.4715,7.7188,7.9297,13.6035,...,83.6014,66.1957,61.5022,83.3939,91.6388,93.7378,97.4809,101.6294,118.1076,POINT (237382.784 2705465.080)
2,X237405Y2705465,237405.139575,2.705465e+06,-7.3403,-8.8226,0.9281,17.5208,7.8016,7.8282,14.2603,...,86.2863,68.8510,64.3726,86.2496,95.1905,96.6428,100.2576,104.1962,120.8167,POINT (237405.140 2705465.403)


In [10]:
max_rows_per_split = 10_000
df_split = np.array_split(df, np.ceil(len(df) / max_rows_per_split))
type(df_split), len(df_split)

(list, 783)

In [11]:
start = time.time()  # Record the start time for performance tracking

select_df = df_split[0]

info_columns = select_df.iloc[:, :3]
cumdisp_df = select_df.iloc[:, 3:-1]

datetime_cols = cumdisp_df.columns.map(
    datetime_handle.convert_to_datetime
)
cumdisp_df.columns = datetime_cols

fulltime_arr = pd.date_range(datetime_cols[0], datetime_cols[-1])

fulltime_df = pd.DataFrame(
    columns=[_date for _date in fulltime_arr if _date not in datetime_cols],
    index=np.arange(len(cumdisp_df)),
)

fulltime_df = fulltime_df.apply(pd.to_numeric, errors='coerce')

fulltime_df = pd.concat([fulltime_df, cumdisp_df], axis=1)

fulltime_cumdisp = fulltime_df[fulltime_arr]

# fulltime_linear_trend_cumdisp = pd.concat([ele[0] for ele in fulltime_cumdisp.apply(analysis.get_linear_trend, axis=1)], axis=1)

cumdisp_average_velocity = pd.DataFrame([ele[1] for ele in fulltime_cumdisp.apply(analysis.get_linear_trend, axis=1)], columns=["velo_mm_day"])
cumdisp_average_velocity["velo_mm_year"] = cumdisp_average_velocity["velo_mm_day"]*365.25

cumdisp_average_velocity = pd.concat([info_columns, cumdisp_average_velocity], axis=1)

# ------------------------------------------------------------------------------
# Performance Logging
# ------------------------------------------------------------------------------
# Record the end time of the process
end = time.time()

# Calculate elapsed time in seconds
ellapse_time_seconds = round((end - start), 2)

# Convert elapsed time to minutes and round to two decimal places
ellapse_time_minutes = round((ellapse_time_seconds / 60), 2)

# Convert elapsed time to hours and round to two decimal places
ellapse_time_hours = round((ellapse_time_seconds / 3600), 2)

# Print the total processing time in seconds
print("Processing time (seconds): " + str(ellapse_time_seconds))

# Print the total processing time in minutes
print("Processing time (minutes): " + str(ellapse_time_minutes))

# Print the total processing time in hours
print("Processing time (hours): " + str(ellapse_time_hours))


Processing time (seconds): 16.57
Processing time (minutes): 0.28
Processing time (hours): 0.0


In [ ]:
def process_by_part(cumdisp_df, datetime_cols):
    